In [1]:
!pip install sympy


In [4]:
from sympy import *
from sympy.printing.str import StrPrinter
from sympy.printing.latex import LatexPrinter

def matrices(names):
    return symbols(names, commutative=False)

d = Function("d", commutative=False)
inv = Function("inv", commutative=False)

class t(Function):
    is_commutative = False
    def __new__(cls, arg):
        if arg.is_Add:
            return Add(*[t(A) for A in arg.args])
        elif arg.is_Mul:
            L = len(arg.args)
            return Mul(*[t(arg.args[L-i-1]) for i in range(L)])
        else:
            return Function.__new__(cls, arg)

MATRIX_DIFF_RULES = {
    Symbol: lambda e, s: d(e) if (e in s) else 0,
    Add: lambda e, s: Add(*[matDiff(arg, s) for arg in e.args]),
    Mul: lambda e, s: Mul(matDiff(e.args[0], s), Mul(*e.args[1:])) + Mul(e.args[0], matDiff(Mul(*e.args[1:]), s)),
    t: lambda e, s: t(matDiff(e.args[0], s)),
    inv: lambda e, s: -e * matDiff(e.args[0], s) * e
}

def matDiff(expr, symbols):
    if expr.__class__ in MATRIX_DIFF_RULES.keys():
        return MATRIX_DIFF_RULES[expr.__class__](expr, symbols)
    else:
        return 0

class matStrPrinter(StrPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '¯¹'
        else:
            return '(' + self._print(expr.args[0]) + ')¯¹'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + "'"

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '∂' + self._print(expr.args[0])
        else:
            return '∂(' + self._print(expr.args[0]) + ')'

def matPrint(m):
    mem = Basic.__str__
    Basic.__str__ = lambda self: matStrPrinter().doprint(self)
    print(str(m).replace('*', ''))
    Basic.__str__ = mem

class matLatPrinter(LatexPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '^{-1}'
        else:
            return '(' + self._print(expr.args[0]) + ')^{-1}'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + '^T'

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '\\partial ' + self._print(expr.args[0])
        else:
            return '\\partial (' + self._print(expr.args[0]) + ')'

def matLatex(expr, profile=None, **kargs):
    if profile is not None:
        profile.update(kargs)
    else:
        profile = kargs
    return matLatPrinter(profile).doprint(expr)

# Define symbols and perform differentiation
X, S = matrices("X S")
H = X * inv(t(X) * inv(S) * X) * t(X) * inv(S)

# Perform differentiation with a list of symbols
symbols_to_diff = [X]  # List of symbols to differentiate with respect to
matPrint(expand(expand(matDiff(H, symbols_to_diff))))
print(matLatex(matDiff(H, symbols_to_diff)))


-Xinv(X'inv(S)X)X'inv(S)d(X)inv(X'inv(S)X)X'inv(S) + Xinv(X'inv(S)X)d(X)'inv(S) - Xinv(X'inv(S)X)d(X)'inv(S)Xinv(X'inv(S)X)X'inv(S) + d(X)inv(X'inv(S)X)X'inv(S)
X \left(- \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} \left(X^T \operatorname{inv}{\left(S \right)} d{\left(X \right)} + d{\left(X \right)}^T \operatorname{inv}{\left(S \right)} X\right) \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} X^T \operatorname{inv}{\left(S \right)} + \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} d{\left(X \right)}^T \operatorname{inv}{\left(S \right)}\right) + d{\left(X \right)} \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} X^T \operatorname{inv}{\left(S \right)}


In [10]:
from sympy import *
from sympy.printing.latex import latex

# Define matrix symbols
X = MatrixSymbol('X', 2, 2)  # Assuming X is a 2x2 matrix
S = MatrixSymbol('S', 2, 2)  # Assuming S is also a 2x2 matrix
d = MatrixSymbol('d', 2, 1)  # d(X) as a 2x1 matrix

# Define the expressions using direct matrix operations
H = X * (t(X) * (S.inv() * X))**-1 * t(X) * S.inv()  # H definition for clarity

# Build the full expression
expr1 = (-X * H * d * H * X * S.inv() +
          X * H * d.T * S.inv() -
          X * H * d.T * X * H * X * S.inv() +
          d * H * X * S.inv())

expr2 = (X * (-H * (t(X) * S.inv() * d + d.T * S.inv() * X) * H * t(X) * S.inv() +
               H * d.T * S.inv()))

# Pretty print the expressions
def pretty_print(expr):
    # Console mode
    print("String Representation:")
    print(expr)

    # LaTeX mode
    print("\nLaTeX Representation:")
    latex_expr = latex(expr)
    print(latex_expr)

pretty_print(expr1)
pretty_print(expr2)


NotImplementedError: noncommutative scalars in MatMul are not supported.

In [11]:
from sympy import symbols, Function, Add, Mul, Basic, expand
from sympy.printing.str import StrPrinter
from sympy.printing.latex import LatexPrinter

def matrices(names):
    return symbols(names, commutative=False)

d = Function("d", commutative=False)
inv = Function("inv", commutative=False)

class t(Function):
    is_commutative = False
    def __new__(cls, arg):
        if arg.is_Add:
            return Add(*[t(A) for A in arg.args])
        elif arg.is_Mul:
            L = len(arg.args)
            return Mul(*[t(arg.args[L-i-1]) for i in range(L)])
        else:
            return super().__new__(cls, arg)

MATRIX_DIFF_RULES = {
    Symbol: lambda e, s: d(e) if (e in s) else 0,
    Add: lambda e, s: Add(*[matDiff(arg, s) for arg in e.args]),
    Mul: lambda e, s: (
        Mul(matDiff(e.args[0], s), Mul(*e.args[1:])) +
        Mul(e.args[0], matDiff(Mul(*e.args[1:]), s))
    ),
    t: lambda e, s: t(matDiff(e.args[0], s)),
    inv: lambda e, s: -e * matDiff(e.args[0], s) * e
}

def matDiff(expr, symbols):
    if expr.__class__ in MATRIX_DIFF_RULES.keys():
        return MATRIX_DIFF_RULES[expr.__class__](expr, symbols)
    else:
        return 0

class matStrPrinter(StrPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '¯¹'
        else:
            return '(' + self._print(expr.args[0]) + ')¯¹'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + "'"

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '∂' + self._print(expr.args[0])
        else:
            return '∂(' + self._print(expr.args[0]) + ')'

def matPrint(m):
    mem = Basic.__str__
    Basic.__str__ = lambda self: matStrPrinter().doprint(self)
    print(str(m).replace('*', ''))
    Basic.__str__ = mem

class matLatPrinter(LatexPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '^{-1}'
        else:
            return '(' + self._print(expr.args[0]) + ')^{-1}'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + '^T'

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '\\partial ' + self._print(expr.args[0])
        else:
            return '\\partial (' + self._print(expr.args[0]) + ')'

def matLatex(expr, profile=None, **kargs):
    if profile is not None:
        profile.update(kargs)
    else:
        profile = kargs
    return matLatPrinter(profile).doprint(expr)

# Define symbols and perform differentiation
X, S = matrices("X S")
H = X * inv(t(X) * inv(S) * X) * t(X) * inv(S)

# Perform differentiation with a list of symbols
symbols_to_diff = [X]  # List of symbols to differentiate with respect to
result = matDiff(H, symbols_to_diff)

# Print results
matPrint(expand(result))
print(matLatex(result))


-Xinv(X'inv(S)X)X'inv(S)d(X)inv(X'inv(S)X)X'inv(S) + Xinv(X'inv(S)X)d(X)'inv(S) - Xinv(X'inv(S)X)d(X)'inv(S)Xinv(X'inv(S)X)X'inv(S) + d(X)inv(X'inv(S)X)X'inv(S)
X \left(- \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} \left(X^T \operatorname{inv}{\left(S \right)} d{\left(X \right)} + d{\left(X \right)}^T \operatorname{inv}{\left(S \right)} X\right) \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} X^T \operatorname{inv}{\left(S \right)} + \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} d{\left(X \right)}^T \operatorname{inv}{\left(S \right)}\right) + d{\left(X \right)} \operatorname{inv}{\left(X^T \operatorname{inv}{\left(S \right)} X \right)} X^T \operatorname{inv}{\left(S \right)}


In [18]:
from sympy import symbols, Function, Add, Mul, Matrix, sin, Symbol, Derivative
from sympy.printing.str import StrPrinter
from sympy.printing.latex import LatexPrinter

def matrices(names):
    return symbols(names, commutative=False)

# Define differentiation function
d = Function("d", commutative=False)
inv = Function("inv", commutative=False)

class t(Function):
    is_commutative = False
    def __new__(cls, arg):
        if arg.is_Add:
            return Add(*[t(A) for A in arg.args])
        elif arg.is_Mul:
            L = len(arg.args)
            return Mul(*[t(arg.args[L-i-1]) for i in range(L)])
        else:
            return super().__new__(cls, arg)

# Differentiation rules
def matDiff(expr, symbols):
    if isinstance(expr, Add):
        return Add(*[matDiff(arg, symbols) for arg in expr.args])
    elif isinstance(expr, Mul):
        return sum(Mul(*[matDiff(arg, symbols) if arg == expr.args[i] else arg for i, arg in enumerate(expr.args)]))
    elif isinstance(expr, t):
        return t(matDiff(expr.args[0], symbols))
    elif isinstance(expr, Symbol) and expr in symbols:
        return d(expr)
    return 0

# Custom string printer for matrices
class matStrPrinter(StrPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '¯¹'
        else:
            return '(' + self._print(expr.args[0]) + ')¯¹'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + "'"

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '∂' + self._print(expr.args[0])
        else:
            return '∂(' + self._print(expr.args[0]) + ')'

def matPrint(m):
    mem = Basic.__str__
    Basic.__str__ = lambda self: matStrPrinter().doprint(self)
    print(str(m).replace('*', ''))
    Basic.__str__ = mem

class matLatPrinter(LatexPrinter):
    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) + '^{-1}'
        else:
            return '(' + self._print(expr.args[0]) + ')^{-1}'

    def _print_t(self, expr):
        return self._print(expr.args[0]) + '^T'

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '\\partial ' + self._print(expr.args[0])
        else:
            return '\\partial (' + self._print(expr.args[0]) + ')'

def matLatex(expr, profile=None, **kargs):
    if profile is not None:
        profile.update(kargs)
    else:
        profile = kargs
    return matLatPrinter(profile).doprint(expr)

# Define symbols and perform differentiation
x1, x2 = symbols('x1 x2')  # Define elements of vector x
X = Matrix([x1, x2])  # Define X as a 2D vector (matrix)
c = Symbol('c')
y1, y2 = symbols('y1 y2')  # Define elements of vector y
y = Matrix([y1, y2])  # Define y as a 2D vector (matrix)
A = Matrix([[1, 0], [0, 1]])  # Example symmetric matrix for A

# Define the expression x' A x + c * sin(y).T * x (element-wise sin)
H = X.T * A * X + c * Matrix([sin(yi) for yi in y]).T * X  # Element-wise sin

# Perform differentiation with respect to x
symbols_to_diff = [X]  # List of symbols to differentiate with respect to
result = matDiff(H, symbols_to_diff)

# Print results
matPrint(expand(result))
print(matLatex(result))


0
0


In [23]:
from sympy import symbols, Matrix, sin, Symbol, diff, expand

# Define the variables and matrices
x1, x2 = symbols('x1 x2')  # Components of vector x
X = Matrix([x1, x2])  # Define X as a 2D vector (matrix)
c = Symbol('c')  # Scalar
y1, y2 = symbols('y1 y2')  # Components of vector y
y = Matrix([y1, y2])  # Define y as a 2D vector (matrix)
A = Matrix([[1, 0], [0, 1]])  # Example symmetric matrix for A

# Define the expression
H = X.T * A * X + c * Matrix([sin(yi) for yi in y]).T * X  # Element-wise sin

# Differentiate the first term (x' A x)
term1_diff = 2 * A * X

# Differentiate the second term (c * sin(y)' * x)
# Using the product rule
sin_y = Matrix([sin(yi) for yi in y])
# Gradient of sin(y)
sin_y_diff = Matrix([diff(sin(yi), yi) for yi in y])
# Combine the derivative terms using product rule
term2_diff = c * (sin_y_diff.T * X + sin_y.T * Matrix([[diff(xi, xj) for xi in X] for xj in X]))

# Combine the results
result = term1_diff + term2_diff

# Print results
print(expand(result))


ShapeError: Matrix size mismatch: (1, 1) + (1, 2).

In [ ]:

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
# Declaration

# -*- coding: utf-8 -*-

#----------------------------------------------------------------------
#
# FUNCTIONS FOR THE AUTOMATIC DIFFERENTIATION  OF MATRICES WITH SYMPY
#
#----------------------------------------------------------------------

from sympy import *
from sympy.printing.str import StrPrinter
from sympy.printing.latex import LatexPrinter



#####  M  E  T  H  O  D  S



def matrices(names):
    ''' Call with  A,B,C = matrix('A B C') '''
    return symbols(names,commutative=False)


# Transformations

d = Function("d",commutative=False)
inv = Function("inv",commutative=False)

class t(Function):
    ''' The transposition, with special rules
        t(A+B) = t(A) + t(B) and t(AB) = t(B)t(A) '''
    is_commutative = False
    def __new__(cls,arg):
        if arg.is_Add:
            return Add(*[t(A) for A in arg.args])
        elif arg.is_Mul:
            L = len(arg.args)
            return Mul(*[t(arg.args[L-i-1]) for i in range(L)])
        else:
            return Function.__new__(cls,arg)


# Differentiation

MATRIX_DIFF_RULES = {

        # e =expression, s = a list of symbols respsect to which
        # we want to differentiate

        Symbol : lambda e,s : d(e) if (e in s) else 0,
        Add :  lambda e,s : Add(*[matDiff(arg,s) for arg in e.args]),
        Mul :  lambda e,s : Mul(matDiff(e.args[0],s),Mul(*e.args[1:]))
                      +  Mul(e.args[0],matDiff(Mul(*e.args[1:]),s)) ,
        t :   lambda e,s : t( matDiff(e.args[0],s) ),
        inv : lambda e,s : - e * matDiff(e.args[0],s) * e
}

def matDiff(expr,symbols):
    if expr.__class__ in MATRIX_DIFF_RULES.keys():
        return  MATRIX_DIFF_RULES[expr.__class__](expr,symbols)
    else:
        return 0



#####  C  O  S  M  E  T  I  C  S


# Console mode

class matStrPrinter(StrPrinter):
    ''' Nice printing for console mode : X¯¹, X', ∂X '''

    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return  self._print(expr.args[0]) +'¯¹'
        else:
            return '(' +  self._print(expr.args[0]) + ')¯¹'

    def _print_t(self, expr):
        return  self._print(expr.args[0]) +"'"

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '∂'+  self._print(expr.args[0])
        else:
            return '∂('+  self._print(expr.args[0]) +')'

def matPrint(m):
    mem = Basic.__str__
    Basic.__str__ = lambda self: matStrPrinter().doprint(self)
    print str(m).replace('*','')
    Basic.__str__ = mem


# Latex mode

class matLatPrinter(LatexPrinter):
    ''' Printing instructions for latex : X^{-1},  X^T, \partial X '''

    def _print_inv(self, expr):
        if expr.args[0].is_Symbol:
            return self._print(expr.args[0]) +'^{-1}'
        else:
            return '(' + self._print(expr.args[0]) + ')^{-1}'
    def _print_t(self, expr):
        return  self._print(expr.args[0]) +'^T'

    def _print_d(self, expr):
        if expr.args[0].is_Symbol:
            return '\partial '+ self._print(expr.args[0])
        else:
            return '\partial ('+ self._print(expr.args[0]) +')'

def matLatex(expr, profile=None, **kargs):
    if profile is not None:
        profile.update(kargs)
    else:
        profile = kargs
    return matLatPrinter(profile).doprint(expr)



#####    T  E  S  T  S


X,S = matrices("X S")
H= X*inv(t(X)*inv(S)*X)*t(X)*inv(S)

matPrint(  expand( expand( matDiff(H,X) ) ) )

print matLatex( matDiff(H,X) )

In [ ]:
https://code.google.com/archive/p/sympy/issues/2759

In [ ]:

sympy
Privacy
 •
Terms
Matrix Symbolic Differentiation
147 views
Subscribe
Mark Yeatman's profile photo
Mark Yeatman
unread,
Jun 8, 2017, 3:15:39 AM
to sympy
I've been digging around trying to figure out the best way to do symbolic matrix differentiation in Sympy. From what, I've read it seems that it exists some what.
The functionality I'm looking for is:

from sympy import MatrixSymbol, Matrix
M = MatrixSymbol('M',2,2)
q = dynamicsymbols('q')

then I'd like to be able to specify that M is a function of q , and then take the total derivative of M with respect to t to get

d(M)/dt = partial(M,q)*partial(q,t) + partial(M,t)

Does this already exist, or not?

Thanks!
Aaron Meurer's profile photo
Aaron Meurer
unread,
Jun 8, 2017, 3:27:01 AM
to sy...@googlegroups.com
I believe we would need https://github.com/sympy/sympy/issues/5855 for this.

Aaron Meurer
> --
> You received this message because you are subscribed to the Google Groups
> "sympy" group.
> To unsubscribe from this group and stop receiving emails from it, send an
> email to sympy+un...@googlegroups.com.
> To post to this group, send email to sy...@googlegroups.com.
> Visit this group at https://groups.google.com/group/sympy.
> To view this discussion on the web visit
> https://groups.google.com/d/msgid/sympy/db580a7c-9b19-4b65-ba57-f7fbdf440716%40googlegroups.com.
> For more options, visit https://groups.google.com/d/optout.